### aluno: **Luiz Felipe da Silva Coelho**

# Trabalho de Banco de Dados II

# Parte 1: inicialização e importações
REFERÊNCIA PRINCIPAL (material em construção): https://docs.google.com/presentation/d/1iIXPOhn2TSm5y5Rf5hiI46GTef1E2gKi/edit?usp=sharing&ouid=105327427325994478755&rtpof=true&sd=true

Vamos juntar o que vocês já enviaram de PI3 com a atividade de revisão que fizemos algumas aulas atrás (link a seguir)
https://docs.google.com/document/d/1bXAUHBfzFwRp2Rsxn4-Afhm_yPm3Pz1DhQtp_CZz-JQ

Crie um novo script do Google Colab, e comece definindo as funções de execução de queries que vimos nas últimas aulas.

In [4]:
import sqlite3
from sqlite3 import Error

## Função de alteração (insert, update, delete...)
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Query {query.split('(')[0].strip()} executada.")
        if cursor.rowcount != -1: print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}'")


# Função de leitura de dados (select)
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

# Parte 2: Tabela do Prisma Ed

Abra a célula de código que corresponde à sua equipe de PI3, nela estará o SQL de criação de tabelas que foi enviado por vocês em atividades anteriores. Copie o código de execução para o seu script.

In [5]:
connection = sqlite3.connect("prisma.sqlite")

query = """
create Table Turma(
  Codigo varchar(10) not null  primary key,
  Curso varchar(20) not null,
  Periodo int not null
);
"""
execute_query(connection, query)

query = """
create Table Aluno(
  RA varchar(7) not null,
  CodigoTurma varchar(10) not null,
  Nome text not null,
  DataNasc date not null,

  primary key(RA, CodigoTurma),
  foreign key(CodigoTurma) references Turma(Codigo) on delete restrict
);
"""
execute_query(connection, query)

query = """
create Table ContatoDoAluno(
  RAAluno varchar(7) not null primary key,
  contato text not null,

  foreign key(RAAluno) references Aluno(RA) on delete cascade
);
"""
execute_query(connection, query)

query = """
create Table Professor(
  ID varchar(8) not null primary key
);
"""
execute_query(connection, query)

query = """
create Table ContatoDoProfessor(
  IDProfessor varchar(8) not null primary key,
  contato text not null,

  foreign key(IDProfessor) references Professor(ID) on delete cascade
);
"""
execute_query(connection, query)

query = """
create Table ProfessorLecionaEmTurma(
  IDProfessor varchar(8) not null,
  CodigoTurma varchar(10) not null,
  Disciplina text not null,

  primary key(IDProfessor, CodigoTurma),
  foreign key(IDProfessor) references Professor(ID) on delete restrict,
  foreign key(CodigoTurma) references Turma(Codigo) on delete restrict
);
"""
execute_query(connection, query)

query = """
create Table Questionario(
  NumeroRef integer not null primary key autoincrement
);
"""
execute_query(connection, query)

query = """
create Table AlunoRespondeQuestionario(
  RAAluno varchar(7) not null,
  NumeroRefQuestionario integer not null,
  DataResposta date not null,

  primary key(RAAluno, NumeroRefQuestionario),
  foreign key(RAAluno) references Aluno(RA) on delete cascade,
  foreign key(NumeroRefQuestionario) references Questionario(NumeroRef) on delete restrict
);
"""
execute_query(connection, query)

query = "INSERT INTO Turma(Codigo, Curso, Periodo) VALUES ('0123456789', 'Informatica','Primeiro Ano');"
execute_query(connection, query)

query = "INSERT INTO Aluno(RA, CodigoTurma, Nome, DataNasc) VALUES ('0123456', '0123456789', 'Maria Lúcia da Silva Rodrigues', '2006-09-17');"
execute_query(connection, query)

query = "INSERT INTO ContatoDoAluno(RAAluno, contato) VALUES ('0123456','nomedoaluno@gmail.com');"
execute_query(connection, query)

query = "INSERT INTO Professor(ID) VALUES ('6543210');"
execute_query(connection, query)

query = "INSERT INTO ContatoDoProfessor(IDProfessor, contato) VALUES ('6543210','nomeprofessor@gmail.com');"
execute_query(connection, query)

query = "INSERT INTO  ProfessorLecionaEmTurma(IDProfessor, CodigoTurma, Disciplina) VALUES ('6543210','0123456789', 'Banco de Dados');"
execute_query(connection, query)

query = "INSERT INTO  AlunoRespondeQuestionario(RAAluno, NumeroRefQuestionario, DataResposta) VALUES ('0123456','2', '2025-05-13');"
execute_query(connection, query)

Query create Table Turma executada.
Query create Table Aluno executada.
Query create Table ContatoDoAluno executada.
Query create Table Professor executada.
Query create Table ContatoDoProfessor executada.
Query create Table ProfessorLecionaEmTurma executada.
Query create Table Questionario executada.
Query create Table AlunoRespondeQuestionario executada.
Query INSERT INTO Turma executada.
1 linha(s) afetadas
Query INSERT INTO Aluno executada.
1 linha(s) afetadas
Query INSERT INTO ContatoDoAluno executada.
1 linha(s) afetadas
Query INSERT INTO Professor executada.
1 linha(s) afetadas
Query INSERT INTO ContatoDoProfessor executada.
1 linha(s) afetadas
Query INSERT INTO  ProfessorLecionaEmTurma executada.
1 linha(s) afetadas
Query INSERT INTO  AlunoRespondeQuestionario executada.
1 linha(s) afetadas


# Atividade 01

Crie duas consultas relevantes para sua aplicação em PI3 que tenham, pelo menos, uma função de agregação em cada. Para cada consulta, descreva a ação esperada (ex.: contar participantes em um evento) e justifique o contexto na aplicação de PI3 (ex.: funcionalidade do back-end para checar se já atingiu o limite de vagas do evento).

CONSULTA 1

Objetivo da consulta (contexto de PI3): Usar esse número para calcular a porcentagem dos estilos de aprendizagem em relação as respostas já existentes;

Descrição da ação esperada ao rodar a consulta: Contar quantos alunos de uma turma já responderam o questionário.


In [6]:
query = """
SELECT t.codigo, COUNT(a.RA)
FROM AlunoRespondeQuestionario r INNER JOIN Aluno a ON r.RAAluno = a.RA
                                 INNER JOIN Turma t ON t.codigo = a.CodigoTurma
GROUP BY t.codigo;
"""
execute_query(connection, query)

Query SELECT t.codigo, COUNT executada.


CONSULTA 2

Objetivo da consulta (contexto de PI3): Contar quantos alunos existem numa turma.

Descrição da ação esperada ao rodar a consulta: Obter o valor referente ao denominador da turma para calculos de porcentagem.

In [7]:
query = """
SELECT t.codigo, COUNT(a.RA)
FROM TURMA t INNER JOIN Aluno a ON t.codigo = a.codigoTurma
GROUP BY t.codigo;
"""
execute_query(connection, query)

Query SELECT t.codigo, COUNT executada.


# Atividade 02

No que seria útil:

- Uma **View**? | Para consultar quantos alunos existem numa turma(já que esse número é o denominador);

- Uma **Procedure**? | Hospedar um select com dados sobre um estilo de aprendizagem fornecido como parâmetro;

- Uma **Function**? | Hospedar um select com dados sobre um estilo de aprendizagem fornecido como parâmetro e retornar se é ou não a maioria na sala;

- Um **Trigger**? | Exibir na tela o estilo predominante de uma turma assim que o professor entrar numa aba específica da turma.